In [70]:
import os 

os.environ["MLFLOW_TRACKING_URL"] ="https://dagshub.com/surani.mayur/datascienceproject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="surani.mayur"
os.environ["MLFLOW_TRACKING_PASSWORD"]="8e0fa2881661d8b0aa394495ae8650f1728eb027"

In [ ]:
import os 

In [ ]:
%pwd

In [ ]:
os.chdir("../")
%pwd

##### Create entity

In [ ]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass()
class ModelEvaluationConfig:
    root_dir: Path 
    test_data_path: Path 
    model_path: Path
    all_params: dict 
    metric_file_name: Path 
    target_column: str 
    mlflow_uri: str 

In [ ]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_dictionaries,save_json

In [ ]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH
                 ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        create_dictionaries([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config= self.config.model_evaluation
        params= self.params.ElasticNet 
        schema= self.schema.TARGET_COLUMN
        
        create_dictionaries([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=(config.root_dir),
            test_data_path=(config.test_data_path),
            model_path=(config.model_path),
            all_params=self.params,
            metric_file_name=(config.metric_file_name),
            target_column=schema.name,
            mlflow_uri='https://dagshub.com/surani.mayur/datascienceproject.mlflow'
        )
        return model_evaluation_config       

##### Model Evaluation

In [ ]:
import os 
import pandas as pd 
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn 
import numpy as np
import joblib

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        # Load test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        # Set up MLflow tracking
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        # Start the run with the experiment ID
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Save metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name, data=scores))

            # Log parameters and metrics to MLflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Log the model
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
                    
            

In [ ]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    print(f"An error occurred: {e}")
    raise e